In [24]:
%pip install beautifulsoup4

  Using cached beautifulsoup4-4.11.1-py3-none-any.whl (128 kB)
  Using cached soupsieve-2.3.2.post1-py3-none-any.whl (37 kB)


2022 World cup groups

In [4]:
import certifi
import ssl
import urllib
import bs4
from bs4 import BeautifulSoup
from urllib.request import build_opener, Request, ProxyHandler, HTTPSHandler
context=ssl.create_default_context(cafile=certifi.where())
https_handler = HTTPSHandler(context=context)
opener = build_opener(https_handler)
urllib.request.install_opener(opener)

In [5]:
import pandas as pd
from string import ascii_uppercase as alphabet
import requests
from urllib import request

web = 'https://en.wikipedia.org/wiki/2022_FIFA_World_Cup'
response = request.urlopen(web)
content = response.read()

all_tables = pd.read_html(content)

# finding tables in group stage
# all_tables[10]
# all_tables[17]
# all_tables[24]
dict_tables = {}
for letter, i in zip(alphabet, range(10, 64, 7)):
    df = all_tables[i]
    df.rename(columns={df.columns[1]: 'Team'}, inplace=True)
    df.pop('Qualification')
    dict_tables[f'Group {letter}'] = df
    
# show all the keys
print(dict_tables.keys())

# show table of Group H
dict_tables

KeyError: 'Qualification'

Historical results

In [27]:
def get_matches(year):
    web = f'https://en.wikipedia.org/wiki/{year}_FIFA_World_Cup'
    response = requests.get(web)
    content = response.text
    soup = BeautifulSoup(content, 'lxml')
    matches = soup.find_all('div', class_='footballbox')

    home = []
    score = []
    away = []

    for match in matches:
        home.append(match.find('th', class_='fhome').get_text())
        score.append(match.find('th', class_='fscore').get_text())
        away.append(match.find('th', class_='faway').get_text())

    dict_football = {'home': home, 'score': score, 'away': away}
    df_football = pd.DataFrame(dict_football)
    df_football['year'] = year
    return df_football

years = [1930, 1934, 1938, 1950, 1954, 1958, 1962, 1966, 1970, 1974,
         1978, 1982, 1986, 1990, 1994, 1998, 2002, 2006, 2010, 2014,
         2018]

# results: historical data
fifa = [get_matches(year) for year in years]
df_fifa = pd.concat(fifa, ignore_index=True)
df_fifa.to_csv("fifa_worldcup_matches.csv", index=False)    

df_fixture = get_matches(2022)
df_fixture.to_csv('fifa_worldcup_fixture.csv', index=False)

In [28]:
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import time
import pandas as pd

from webdriver_manager.chrome import ChromeDriverManager

options = webdriver.ChromeOptions()
options.binary_location = r"C:\Program Files (x86)\Google\Chrome\Application\chrome.exe"
chrome_driver_binary = r"C:/Users/reegan.alward/.wdm/chromedriver/75.0.3770.8/win32/chromedriver.exe"
driver = webdriver.Chrome(chrome_driver_binary, chrome_options=options)

path = "/mnt/c/Users/reegan.alward/AppData/Local/Temp/1/Temp1_chromedriver_win32.zip"
service = Service(executable_path=path)
driver = webdriver.Chrome(service=service)


def get_misssing_data(year):
    web = f'https://en.wikipedia.org/wiki/{year}_FIFA_World_Cup'

    driver.get(web)
    matches = driver.find_elements(by='xpath', value='//td[@align="right"]/.. | //td[@style="text-align:right;"]/..')
    # matches = driver.find_elements(by='xpath', value='//tr[@style="font-size:90%"]')

    home = []
    score = []
    away = []

    for match in matches:
        home.append(match.find_element(by='xpath', value='./td[1]').text)
        score.append(match.find_element(by='xpath', value='./td[2]').text)
        away.append(match.find_element(by='xpath', value='./td[3]').text)

    dict_football = {'home': home, 'score': score, 'away': away}
    df_football = pd.DataFrame(dict_football)
    df_football['year'] = year
    time.sleep(2)
    return df_football


years = [1930, 1934, 1938, 1950, 1954, 1958, 1962, 1966, 1970, 1974,
         1978, 1982, 1986, 1990, 1994, 1998, 2002, 2006, 2010, 2014,
         2018]

fifa = [get_misssing_data(year) for year in years]
driver.quit()
df_fifa = pd.concat(fifa, ignore_index=True)
df_fifa.to_csv("fifa_worldcup_missing_data.csv", index=False)

WebDriverException: Message: unknown error: cannot find Chrome binary
Stacktrace:
#0 0x5637046ad2a3 <unknown>
#1 0x56370446bf77 <unknown>
#2 0x563704492047 <unknown>
#3 0x5637044907d0 <unknown>
#4 0x5637044d10b7 <unknown>
#5 0x5637044d0a5f <unknown>
#6 0x5637044c8903 <unknown>
#7 0x56370449bece <unknown>
#8 0x56370449cfde <unknown>
#9 0x5637046fd63e <unknown>
#10 0x563704700b79 <unknown>
#11 0x5637046e389e <unknown>
#12 0x563704701a83 <unknown>
#13 0x5637046d6505 <unknown>
#14 0x563704722ca8 <unknown>
#15 0x563704722e36 <unknown>
#16 0x56370473e333 <unknown>
#17 0x7f1eba312609 start_thread


In [ ]:
df_fixture['home'] = df_fixture['home'].str.strip()
df_fixture['away'] = df_fixture['away'].str.strip()

In [ ]:
df_fifa.drop_duplicates(inplace=True)
df_fifa.sort_values('year', inplace=True)
df_fifa